# 2. Feature Selection

#### References  
https://www.kaggle.com/competitions/amex-default-prediction/discussion/335892

In [1]:
%matplotlib inline
import os
from pathlib import Path
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix, parallel_coordinates
import seaborn as sns
import matplotlib.pylab as plt
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
train_r = pd.read_parquet("C:/Users/kimch/Desktop/American Express/train.parquet")

In [4]:
labels = pd.read_csv("C:/Users/kimch/Desktop/American Express/train_labels.csv")

In [5]:
train = train_r.merge(labels, right_on = 'customer_ID', left_on = "customer_ID")

In [6]:
train["S_2"] = pd.to_datetime(train.S_2, format="%Y-%m-%d")

In [7]:
train_p = train.drop(columns=["D_88", "B_42", "D_110", "D_73", "B_39", "D_134", "B_29", "D_132", "D_76", "D_42", "D_142", "D_53", "B_17",
                              "D_50", "D_105", "D_56", "S_9"])

In [8]:
train_p = train_p.dropna(subset=["P_3", "D_69", "D_115", "D_118", "D_119", "D_121", "D_55", "D_131", "D_141", "D_130", "D_104", "R_27", "D_128", "D_102", "B_13", "D_133", "D_144", "D_52", "P_2"])

In [10]:
train_fillna = train_p.set_index('S_2')
train_fillna = train_fillna.interpolate(method="time")

In [11]:
train_fillna_drop = train_fillna.drop(columns=['S_7', "S_24", "B_11", "B_37", "B_33", "B_18", "B_23", "B_37", "B_15", "B_20"])

In [12]:
missing = pd.DataFrame(columns=['column', 'null_sum'])

for col in train_fillna_drop.columns:
    n = train_fillna_drop[col].isnull().sum()
    missing =  missing.append(pd.DataFrame([[col, n]], columns=['column', 'null_sum']), ignore_index=True)

missing.set_index("column", inplace=True)
missing = missing.sort_values(by="null_sum", ascending=False)
missing[missing["null_sum"]>0]

,null_sum
column,
D_43,19
D_77,11


In [13]:
train_nonull = train_fillna_drop.dropna(subset=["D_43", "D_77"])

# 변수 선택 (Feature Selecting)

변수 선택 방법 (https://subinium.github.io/feature-selection/)  
1. 필터 기법  
2. 래퍼 기법 (전진, 후방, 단계적)
3. 임베디드 (L1, L2)    
  
+feature_importancs_ : tree 기법에서 중요도 파악

In [ ]:
# RandomForest, GradientBoosting -> memory error

In [14]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

In [15]:
train_nonull_n = train_nonull.drop(columns=["customer_ID"])

In [16]:
train_X = train_nonull_n.drop(columns=["target"])
train_y = train_nonull_n["target"]

In [17]:
from sklearn.feature_selection import SelectFromModel

In [ ]:
embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), threshold="1.25*median")
embeded_rf_selector.fit(train_X, train_y)